# Google Pub/Sub Streaming Consumer

In [2]:
# Imports the Google Cloud client library
from google.cloud import pubsub
from boto import kinesis
import datetime
import time
import dateutil.parser

STREAM_NAME="latency"
PROJECT="streaming-154400"

# Instantiates a client
pubsub_client = pubsub.Client(project=PROJECT)

# Prepares the new topic
topic = pubsub_client.topic(STREAM_NAME)

In [ ]:
pubsub_client

In [ ]:
try:
    subscription = topic.subscription("Default")
    subscription.delete()        
except: 
    pass #subscription already exists

In [ ]:
run_timestamp=datetime.datetime.now()
RESULT_FILE= "results/google-pub-sub-latency-" + run_timestamp.strftime("%Y%m%d-%H%M%S") + ".csv"
try:
    os.makedirs("results")
except:
    pass
output_file=open(RESULT_FILE, "w")

In [ ]:
subscription = topic.subscription("Default")
try:
    subscription.create()        
except: 
    pass # subscription already exists

while True:
    pulled = subscription.pull(return_immediately=False, max_messages=10000)
    print "Pulled %d messages"%len(pulled)
    for t in pulled: 
        #print t
        message = t[1].data
        subscription.acknowledge([t[0]])
        #print message
        now = time.time()
        #sent_time=datetime.datetime.strptime(message.value, "%Y-%m-%dT%H:%M:%S.%fZ")
        sent_time_string = message.split(";")[0]
        sleep_time =float(message.split(";")[1])
        sent_time = dateutil.parser.parse(sent_time_string)
        sent_time_ts = time.mktime(sent_time.timetuple())
        lat = now-sent_time_ts    
        result = "google, latency, 0, %.5f, %.5f\n"%(1/sleep_time, lat)
        print(result)
        output_file.write(result)
        output_file.flush()